In [1]:
!huggingface-cli whoami

ledengary


In [2]:
from transformers import AutoTokenizer
import transformers
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import openai
import json
import time
import pickle
import re

/home/admin/VS Code Projects/HFH/Symptom Extraction/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import numpy as np
import random
import os

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU setups
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

the_seed = 42
seed_everything(the_seed)

# Mixtral

In [4]:
# !pip install --upgrade transformers
# !pip install --upgrade tokenizers

In [5]:
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)
model.eval()

Loading checkpoint shards: 100%|██████████| 19/19 [00:42<00:00,  2.24s/it]


MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=14336, bias=False)
              (w2): Linear(in_features=14336, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=14336, bias=False)
        

In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

In [7]:
from transformers import pipeline

mixtral_pipeline = pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=False,  # if using langchain set True
    task="text-generation",
    # we pass model parameters here too
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    repetition_penalty=1.1,  # if output begins repeating increase
    device_map="auto"
)
mixtral_pipeline

In [8]:
def get_mixtral_response(prompt, top_k=10, top_p=0.9, temperature=0.9, max_length=4000):
    sequences = mixtral_pipeline(
        prompt,
        do_sample=True,
        top_k=top_k,
        return_full_text=False,
        num_return_sequences=1,
        temperature=temperature, 
        top_p=top_p,
        eos_token_id=tokenizer.eos_token_id,
        max_length=max_length,
        pad_token_id=mixtral_pipeline.tokenizer.eos_token_id
    )
    return sequences[0]['generated_text']

In [15]:
target_toxicity = 'Urinary retention'
target_note = """This 71-year-old retired painter for Parvason Communications, who built a home in stanford of neurology,  reports posterior neck pain of 1 weeks duration.  this began in the absence of any trauma.  The pain is bilateral on left posterolateral left and posterolateral right pad of the neck and radiates to the proximal shoulder.  There is no radicular pain down the arms there is no loss of sensation down the arms.  Pain is elicited by rotating the head to the left or to the right.    Records from other physicians were reviewed the patient has been seeing a podiatrist for foot pain.  His GI specialists have ruled out cancer as an underlying cause of his iron deficiency.          CURRENT MEDICATIONS:.  1. Aspirin 81 mg q.d.  2. Eplerone 50 mg QD.  3. Lasix 40 mg BID.  4. Lipitor 80 mg q.d.  5. Toprol-XL 50 mg BID.  6. Omeprazole 20 mg BID---taken prn (he is not taking it now).  7. Zantac 150 mg QHS.  8. Finasteride 5 mg QD.  9. Tylenol #3 BID prn----starting today..  10. Lupron 30 mg IM---given  12/21/2013.  11. Sildenafil 20 3-5 tabs prn.  12. Depo-Provera  13. Fosamax 70 mg Q Week.--------Started 10/30/12.---he stopped taking this before 1 year.--restarted  2013/21/12      ALLERGIES:  1. Prinivil caused a cough.  2. Antivert was temporarily held because of his tendency towards urinary retention,  but he is not truly allergic to it.  3. Imdur caused a headache, but was not a true allergy.  4. Keflex is listed as causing an unspecified reaction, but I cannot verify whether  this is a true allergy.  5. Cozaar was held after years of use when he developed lip angioedema-------angioedema continued after stopping this drug-------a questionable association-..  6. Quinolones---relatively contraindicated given his aneurysms.    PAST MEDICAL HISTORY:      Coronary artery disease, triple-vessel, with a very long LAD  lesion, medical management preferred. Stress-echo 22-01-2012 showed 7 mets, ischemic EKG changes but no wall motion changes.  Echocardiogram 1-21 shows a patent foramen ovale with right-to-left shunt.   Dobutamine echocardiogram in 1-22-2003 showed  normal valves, ejection fraction 63%, LAD ischemia.      Hypertension.       MRI in 1997/01/21 showed a lacunar stroke,      syncopal episodes with a positive tilt table test in 2005/21/01,with a repeat episode of syncope a few years ago and 12/22/2007..        Prostate cancer---Gleason 8, PSA=11.9 in 2005-22-11. Lupron  In progress.. Abd-Pelvic CT 11/22/2005 shows no metastases, renal stones.  Radiation therapy to prostate completed.  MRI 3-22-2005 shows no extracapsular spread, Bone scan 01/22/05 shows  skull lesion---Head CT shows stable but expansile lytic lesion parietal area 4-22-06------f/u PSA = 0 in 12-22-2007.  . Osteopenia from treatment----bone density T-2.0, Z-0.9---Fosamax started 12-22-2007.      Hereditary Hemorrhagic Telangiectasia with AV malformation of the lung and recurrent epistaxis.   Thoracic CT 12-22-2007 (pulmonary embolism protocol) shows  LLL A-V malformation,----embolization pending.       Thyroid nodules, --thyroid U/S shows complex nodules---  bx benign 22-12----f/u due 22-12.       Ascending aortic aneurysm--4.4 cm----next CT due 8-22-2007.  Splenic artery aneurysm.     Renal stones---Left hydronephrosis --- ESWL--failed---ureterascopy with stone retrieval pending.    CT renal stone protocol 1-22-07 shows left hydronephrosis, pulmonary AV malformation, abd aortic aneurysm 2007-02-22.     Iron deficiency-- colonoscopy  tubular adenomas in 2007 january 22nd, EGD with biopsy shows nonbleeding AVMs. Enteroscopy---non-bleeding AVMs, 2019.  next colonoscopy due in 2012-01-22.      Fasting blood sugar 112/hemoglobin A1c 6.1.      L4-L5  nerve root compression, status post laminectomy with residual left footdrop,       Venous Doppler negative in 1-22-2022.       Hyperlipidemia,    right inguinal hernia, recurrent urinary tract obstruction with right epididymo-orchitis with Pseudomonas, treated with TURP, cystoscopy otherwise negative in 1-22-2028.     Severe sleep apnea--CPAP used..  Angioedema---anti-IgE fc receptor antibody negative.    Pharyngeal abscess with sepsis.    PAST SURGICAL HISTORY: Appendectomy, hernia repair, L4-L5 laminectomy, TURP. No  surgical bleeding complications,or deep venous thromboses during any of his  previous operations.    SOCIAL HISTORY: He is retired as a painter for Parvason Communications. Married to Hoj, Psychiatry. Two  daughters, one son. He came in to this country from PHARMACY OF JFK in 35-01-22. Smoked, but  quit 23 years ago. Drinks alcohol occasionally. No illegal drugs.  Daughter is Penny and Emmanuelle.  Lives in STANFORD HOSPITAL  in a few months each year.    FAMILY HISTORY: Mother died with coronary artery disease and dementia at age 89.   Mother's siblings have late onset dementia.   Father  died with a lacunar stroke in his 70s. The patient has one brother with a possible  CABG at age 59. He has two sisters, one of whom has unspecified heart disease  after possible syncope a few years ago and now has a pacemaker.   2nd sister is healthy. Daughter has nosebleeds and AV malformation----strong suspicion for Hereditary Hemorrhagic Telangiectasia.  Grandson has nosebleeds.    There is no cancer,  diabetes, deep venous thromboses, aneurysms,  in the family.     Physical exam---limited because this was a telephone visit but the patient did not sound in significant distress but he verbally reported pain with rotating his head to the left into the right.    IMPRESSION     musculoskeletal posterior neck pain.  Tylenol No.  3 1 tab b.i.d. as needed.  If there is not substantial improvement within 1 week, I made him promise that he would make an appointment to see me."""
mixtral_prompt = """Review a single clinical note and locate the sentence where the symptom of """ + target_toxicity + """ is present. Now go step-by-step and tell me if """ + target_toxicity +""" is present. You must respond `"YES"`or `"NO"` in JSON format.
Use these rules:
- Return "YES" if """ + target_toxicity + """ is present.
- Return "NO" if """ + target_toxicity + """ is absent.
"""

prompt_add_on = f"""
Progress Note: 
{target_note}"""
final_prompt = '<s> [INST] ' +  mixtral_prompt + '[/INST]\n' + prompt_add_on + '\nAssistant: ```json\n{\n"label": '
student_temp = 0.2
student_top_p = 0.1
student_top_k = 1
print(final_prompt)
rsp = get_mixtral_response(final_prompt, top_k=1, top_p=0.1, temperature=0.2)
print('Response:', rsp)

<s> [INST] Review a single clinical note and locate the sentence where the symptom of Urinary retention is present. Now go step-by-step and tell me if Urinary retention is present. You must respond `"YES"`or `"NO"` in JSON format.
Use these rules:
- Return "YES" if Urinary retention is present.
- Return "NO" if Urinary retention is absent.
[/INST]

Progress Note: 
This 71-year-old retired painter for Parvason Communications, who built a home in stanford of neurology,  reports posterior neck pain of 1 weeks duration.  this began in the absence of any trauma.  The pain is bilateral on left posterolateral left and posterolateral right pad of the neck and radiates to the proximal shoulder.  There is no radicular pain down the arms there is no loss of sensation down the arms.  Pain is elicited by rotating the head to the left or to the right.    Records from other physicians were reviewed the patient has been seeing a podiatrist for foot pain.  His GI specialists have ruled out cancer as an